In [1]:
import pandas as pd
import numpy as np
import matplotlib
import sys

from readCanLitefileUtils import *
from calibUtils import *

In [2]:
# define the inputs

#file 1 (DACset 0x000)
#file_DACset000 = './files/21-05-06_160653_calib_SN04_DACSet000_20x.csv'
#file_DACset000 = './files/SN11/21-05-20_110216_DACset000_SN11_20x.csv'
file_DACset000 = './files/SN11/21-05-20_110216_DACset000_SN11_20x_v2.csv'



#file 2 (DACset 0xFFF)
#file_DACsetFFF = './files/21-05-06_160653_calib_SN04_DACSetFFF_20x.csv'
#file_DACsetFFF = './files/SN11/21-05-20_110216_DACsetFFF_SN11_20x.csv'
file_DACsetFFF = './files/SN11/21-05-20_110216_DACsetFFF_SN11_20x_v2.csv'



DMMmin = [108.718, 108.744, 108.752, 108.746, 108.739, 108.722, 
          108.736, 108.699, 108.750, 108.747, 108.743, 108.732]

DMMmax = [116.288, 116.275, 116.328, 116.291, 116.273, 116.282,
          116.311, 116.267, 116.327, 116.316, 116.219, 116.250]


file_HVoff = './files/SN11/21-05-20_110216_HVoff_SN11_20x.csv'

DMMHVmin = 1.305
DMMHVmax = 120.
#data for HV calibration
#data_HV = {'ADCmin': [0x02EB], 'DMMmin': [1.378], 'ADCmax': [0xFEB5], 'DMMmax': [120]}
#df_HV = pd.DataFrame(data=data_HV)

def GetADCvalues(file_DACset):

    tmp_df = pd.read_csv(file_DACset)

    #remove some columns not exploitable
    tmp_df.drop(['Bus','No', 'ID (hex)', 'Message', 'ASCII'], inplace=True, axis=1)

    #save dataframe in a file to be read by the script
    filename = file_DACset.split('/')[2]
    tmp_filename = 'calibfile_fromscript_'+ filename+'.txt'
    tmp_df.to_csv(tmp_filename, sep=',', index=False, header=False, quotechar = ' ')

    #open file to read
    infile = open(tmp_filename, 'r')

    #call function which read the file and return the content ordered in a list of dictionaries
    d_list = readfile(infile)

    DAC_set = getDACsetperchannel(d_list)
    DAC_set

    my_allch_list = getADCperchannel(d_list)
    mych_average_biasV = Getch_average(my_allch_list)
    hexprint = list(map(trunchex, mych_average_biasV))

    HV = getHV(d_list)

    return DAC_set, hexprint

In [3]:
DACset000, ADCmin, HVmin = GetADCvalues(file_DACset000)
print('-----')
print(DACset000)
print(ADCmin)
print(HVmin)

DACsetFFF, ADCmax, HVmax = GetADCvalues(file_DACsetFFF)
print(DACsetFFF)
print(ADCmax)
print(HVmax)



-----
0x0
['e6b0', 'e6ce', 'd78a', 'e6c7', 'e6ce', 'e6c0', 'e6cc', 'e6bb', 'e6df', 'd702', 'e6ea', 'e6d4']
0xfd8c
0xfff
['f6bc', 'f6de', 'e55c', 'f6dd', 'f6e2', 'f6d1', 'f6e0', 'f6cc', 'f6f2', 'e4e2', 'f701', 'f6e7']
0xfd78


In [4]:
#the HVmin value is overwitten by the good one which comes from the file with HV OFF

HVmin = GetHVvalue(file_HVoff)

In [5]:
#data for HV calibration
data_HV = {'ADCmin': [HVmin], 'DMMmin': [DMMHVmin], 'ADCmax': [HVmax], 'DMMmax': [DMMHVmax]}
df_HV = pd.DataFrame(data=data_HV)
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax
0,0x259,1.305,0xfd78,120.0


In [6]:
DACset000 = '0x0'
DACsetFFF = '0xfff'
#tmp fix for SN11

In [7]:
#build the dataframe to use to extract the calibration parameters
if (DACset000 == '0x0'):
    col0 = 'ADCmin'
else: 
    #print('Error! not the expected values for the DACset')
    sys.exit('Error! not the expected values for the DACset')


if (DACsetFFF == '0xfff'):
    col3 = 'ADCmax'
else: 
    sys.exit('Error! not the expected values for the DACset')
    

df_out = pd.DataFrame(list(zip(ADCmin, DMMmin, ADCmax, DMMmax)), columns = [col0, 'DMMmin' ,  col3, 'DMMmax'])
df_out

,ADCmin,DMMmin,ADCmax,DMMmax
0,e6b0,108.718,f6bc,116.288
1,e6ce,108.744,f6de,116.275
2,d78a,108.752,e55c,116.328
3,e6c7,108.746,f6dd,116.291
4,e6ce,108.739,f6e2,116.273
5,e6c0,108.722,f6d1,116.282
6,e6cc,108.736,f6e0,116.311
7,e6bb,108.699,f6cc,116.267
8,e6df,108.750,f6f2,116.327
9,d702,108.747,e4e2,116.316


In [8]:
#convert ADCmin and ADCmax in decimal numbers
df_out = hex_to_dec(df_out)
df_out

,ADCmin,DMMmin,ADCmax,DMMmax
0,59056,108.718,63164,116.288
1,59086,108.744,63198,116.275
2,55178,108.752,58716,116.328
3,59079,108.746,63197,116.291
4,59086,108.739,63202,116.273
5,59072,108.722,63185,116.282
6,59084,108.736,63200,116.311
7,59067,108.699,63180,116.267
8,59103,108.750,63218,116.327
9,55042,108.747,58594,116.316


In [9]:
# compute gain and offset for the Bias Voltages
# a_coeff = 1.8535e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )

df_out = biasV_params(df_out)
df_out

,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV
0,59056,108.718,63164,116.288,0.994198,-58.173844
1,59086,108.744,63198,116.275,0.988114,289.292524
2,55178,108.752,58716,116.328,1.155286,-4390.701162
3,59079,108.746,63197,116.291,0.988508,273.687608
4,59086,108.739,63202,116.273,0.987547,320.653040
5,59072,108.722,63185,116.282,0.991678,77.945238
6,59084,108.736,63200,116.311,0.992921,-0.518020
7,59067,108.699,63180,116.267,0.992727,7.919001
8,59103,108.750,63218,116.327,0.993424,-41.861027
9,55042,108.747,58594,116.316,1.149669,-4008.925089


In [10]:
# Gain biasV = math.ceil(Gain BiasV* 32768)
# Offset biasV = round(offset BiasV)  

df_out = code_offsetandgain(df_out,'biasV')
df_out

,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV
0,59056,108.718,63164,116.288,0.994198,-58.173844,32578,-58
1,59086,108.744,63198,116.275,0.988114,289.292524,32379,289
2,55178,108.752,58716,116.328,1.155286,-4390.701162,37856,-4391
3,59079,108.746,63197,116.291,0.988508,273.687608,32391,274
4,59086,108.739,63202,116.273,0.987547,320.653040,32360,321
5,59072,108.722,63185,116.282,0.991678,77.945238,32495,78
6,59084,108.736,63200,116.311,0.992921,-0.518020,32536,-1
7,59067,108.699,63180,116.267,0.992727,7.919001,32530,8
8,59103,108.750,63218,116.327,0.993424,-41.861027,32553,-42
9,55042,108.747,58594,116.316,1.149669,-4008.925089,37672,-4009


In [11]:
# convert following prescriptions
 # GainUsig BiasV   = hex(Gain_Usig BiasV) 
 # Offset_Sig BiasV = tohextwocompl(Offset_Sig BiasV ,15) 

df_out = convert_to_exadec(df_out, 'biasV')
df_out

,ADCmin,DMMmin,ADCmax,DMMmax,Gain BiasV,offset BiasV,Gain_Usig BiasV,Offset_Sig BiasV,Gain_hex BiasV,Offset_hex BiasV
0,59056,108.718,63164,116.288,0.994198,-58.173844,32578,-58,0x7f42,0xffc6
1,59086,108.744,63198,116.275,0.988114,289.292524,32379,289,0x7e7b,0x121
2,55178,108.752,58716,116.328,1.155286,-4390.701162,37856,-4391,0x93e0,0xeed9
3,59079,108.746,63197,116.291,0.988508,273.687608,32391,274,0x7e87,0x112
4,59086,108.739,63202,116.273,0.987547,320.653040,32360,321,0x7e68,0x141
5,59072,108.722,63185,116.282,0.991678,77.945238,32495,78,0x7eef,0x4e
6,59084,108.736,63200,116.311,0.992921,-0.518020,32536,-1,0x7f18,0xffff
7,59067,108.699,63180,116.267,0.992727,7.919001,32530,8,0x7f12,0x8
8,59103,108.750,63218,116.327,0.993424,-41.861027,32553,-42,0x7f29,0xffd6
9,55042,108.747,58594,116.316,1.149669,-4008.925089,37672,-4009,0x9328,0xf057


In [12]:
#do the same for the HV
#Compute gain and offset for the HV

# a_coeff = 1.8409e-3
# b_coeff = 0
# Gain BiasV = (DMMmax - DMMmin) / (ADCmax - ADCmin ) * 1/a_coeff
# offset BiasV = ADCmin - (DMMmin - b_coeff )/ (Gain BiasV * a_coeff )

df_HV = hex_to_dec(df_HV)
df_HV = HV_params(df_HV)
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV
0,601,1.305,64888,120.0,1.00295,105.807658


In [13]:
# Gain HV = math.ceil(Gain HV* 32768)
# Offset HV = round(offset HV)  

df_HV = code_offsetandgain(df_HV,'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV
0,601,1.305,64888,120.0,1.00295,105.807658,32865,106


In [14]:
# convert following prescriptions
# GainUsig BiasV   = hex(Gain_Usig HV) 
# Offset_Sig BiasV = tohextwocompl(Offset_Sig HV ,15) 

df_HV = convert_to_exadec(df_HV, 'HV')
df_HV

,ADCmin,DMMmin,ADCmax,DMMmax,Gain HV,offset HV,Gain_Usig HV,Offset_Sig HV,Gain_hex HV,Offset_hex HV
0,601,1.305,64888,120.0,1.00295,105.807658,32865,106,0x8061,0x6a


In [15]:
df_calib_biasV = df_out[['Gain_hex BiasV','Offset_hex BiasV'] ].copy()
df_calib_biasV.to_csv('calibDB/calib_biasV_SN11_v2.csv')
print(df_calib_biasV)

print('\n\n\n')
df_calib_HV = df_HV[['Gain_hex HV','Offset_hex HV'] ].copy()
df_calib_HV.to_csv('calibDB/calib_HV_SN11_v2.csv')
print(df_calib_HV)

   Gain_hex BiasV Offset_hex BiasV
0          0x7f42           0xffc6
1          0x7e7b            0x121
2          0x93e0           0xeed9
3          0x7e87            0x112
4          0x7e68            0x141
5          0x7eef             0x4e
6          0x7f18           0xffff
7          0x7f12              0x8
8          0x7f29           0xffd6
9          0x9328           0xf057
10         0x7d57            0x31f
11         0x7e2b            0x1a7




  Gain_hex HV Offset_hex HV
0      0x8061          0x6a


In [16]:
df_calib_biasV_tocheck = df_out[['Gain_Usig BiasV', 'Offset_Sig BiasV', 'Gain_hex BiasV','Offset_hex BiasV']].copy()
df_calib_biasV_tocheck.to_csv('calibDB/calib_biasV_SN11_v2_forcheck.csv')